In [2]:
import numpy as np
import pandas as pd
from obspy import read
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import os
import dask.dataframe as dd

In [3]:
cat_directory = '../data/lunar/training/catalogs/'
cat_file = cat_directory + 'apollo12_catalog_GradeA_final.csv'
cat = pd.read_csv(cat_file)
cat

,filename,time_abs(%Y-%m-%dT%H:%M:%S.%f),time_rel(sec),evid,mq_type
0,xa.s12.00.mhz.1970-01-19HR00_evid00002,1970-01-19T20:25:00.000000,73500.0,evid00002,impact_mq
1,xa.s12.00.mhz.1970-03-25HR00_evid00003,1970-03-25T03:32:00.000000,12720.0,evid00003,impact_mq
2,xa.s12.00.mhz.1970-03-26HR00_evid00004,1970-03-26T20:17:00.000000,73020.0,evid00004,impact_mq
3,xa.s12.00.mhz.1970-04-25HR00_evid00006,1970-04-25T01:14:00.000000,4440.0,evid00006,impact_mq
4,xa.s12.00.mhz.1970-04-26HR00_evid00007,1970-04-26T14:29:00.000000,52140.0,evid00007,deep_mq
...,...,...,...,...,...
71,xa.s12.00.mhz.1974-10-14HR00_evid00156,1974-10-14T17:43:00.000000,63780.0,evid00156,impact_mq
72,xa.s12.00.mhz.1975-04-12HR00_evid00191,1975-04-12T18:15:00.000000,65700.0,evid00191,impact_mq
73,xa.s12.00.mhz.1975-05-04HR00_evid00192,1975-05-04T10:05:00.000000,36300.0,evid00192,impact_mq
74,xa.s12.00.mhz.1975-06-24HR00_evid00196,1975-06-24T16:03:00.000000,57780.0,evid00196,impact_mq


In [74]:
quake_timestamps = cat["time_abs(%Y-%m-%dT%H:%M:%S.%f)"].values
quake_timestamps_pandas = [pd.to_datetime(ts) for ts in quake_timestamps] 

In [5]:
mseed_folder_path = r'..\data\lunar\training\data\S12_GradeA'
lunar_data_names = cat['filename'].values

lunar_data_file_path = [os.path.join(mseed_folder_path, f'{file_name}.csv') for file_name in lunar_data_names]
lunar_data_file_path = [file_path for file_path in lunar_data_file_path if os.path.isfile(file_path)]

In [6]:
lunar_data_file_path

['..\\data\\lunar\\training\\data\\S12_GradeA\\xa.s12.00.mhz.1970-01-19HR00_evid00002.csv',
 '..\\data\\lunar\\training\\data\\S12_GradeA\\xa.s12.00.mhz.1970-03-25HR00_evid00003.csv',
 '..\\data\\lunar\\training\\data\\S12_GradeA\\xa.s12.00.mhz.1970-03-26HR00_evid00004.csv',
 '..\\data\\lunar\\training\\data\\S12_GradeA\\xa.s12.00.mhz.1970-04-25HR00_evid00006.csv',
 '..\\data\\lunar\\training\\data\\S12_GradeA\\xa.s12.00.mhz.1970-04-26HR00_evid00007.csv',
 '..\\data\\lunar\\training\\data\\S12_GradeA\\xa.s12.00.mhz.1970-06-15HR00_evid00008.csv',
 '..\\data\\lunar\\training\\data\\S12_GradeA\\xa.s12.00.mhz.1970-06-26HR00_evid00009.csv',
 '..\\data\\lunar\\training\\data\\S12_GradeA\\xa.s12.00.mhz.1970-07-20HR00_evid00010.csv',
 '..\\data\\lunar\\training\\data\\S12_GradeA\\xa.s12.00.mhz.1970-07-20HR00_evid00011.csv',
 '..\\data\\lunar\\training\\data\\S12_GradeA\\xa.s12.00.mhz.1970-09-26HR00_evid00013.csv',
 '..\\data\\lunar\\training\\data\\S12_GradeA\\xa.s12.00.mhz.1970-10-24HR00_evid

Defining reading functions

In [8]:
df = pd.read_csv(lunar_data_file_path[0])

In [14]:
df.columns

Index(['time_abs(%Y-%m-%dT%H:%M:%S.%f)', 'time_rel(sec)', 'velocity(m/s)'], dtype='object')

Defining Processing functions

In [32]:
def reducing_rows_by_mean_average(df, n_parameters, target_column):
    """
    Preprocesses a DART dataset by averaging every n_parameters rows in the 
    target column, while keeping all other columns and reducing them 
    based on the target column's grouping.

    Args:
        df (pandas.DataFrame): The input DART dataset.
        n_parameters (int): The number of rows to average.
        target_column (str): The name of the column to preprocess.

    Returns:
        pandas.DataFrame: The processed dataset with averaged values in the
                         target column and other columns reduced accordingly.
    """

    # Create a new column to group the data for averaging
    df['group'] = df.index // n_parameters

    # Calculate the mean for each group in the target column
    processed_df = df.groupby('group').agg({
        target_column: 'mean',  # Average the target column
        **{col: 'first' for col in df.columns if col != target_column}  # Keep the first value for other columns
    })

    # Reset the index and drop the 'group' column
    processed_df = processed_df.reset_index(drop=True)  # drop=True to avoid the error
    processed_df = processed_df.drop(columns=["group"])

    return processed_df

def preprocess_data_rms(df, window_size=4):
  """
  Preprocesses a dataframe by calculating the absolute root mean square (RMS) 
  over a rolling window.

  Args:
      df (pandas.DataFrame): The input dataframe with a 'data' column.
      window_size (int, optional): The size of the rolling window in samples. 
                                  Defaults to 10.

  Returns:
      pandas.DataFrame: A new dataframe with the 'rms' column containing the 
                       calculated RMS values.
  """

  # Calculate the absolute RMS using a rolling window
  df['Value'] = df['Value'].abs().rolling(window=window_size, center=True).mean()

  # Drop the initial and final rows with NaN values due to the rolling window
  df = df.dropna()

  return df

In [58]:
def preprocess_df(df):
    # Moon
    df = df.rename(
        columns={
            "time_abs(%Y-%m-%dT%H:%M:%S.%f)": "AbsTime",
            "time_rel(sec)": "RelTime",
            "velocity(m/s)": "Value",
        }
    )
    # Mars
    df = df.rename(
    columns={
        "time(%Y-%m-%dT%H:%M:%S.%f)": "AbsTime",
        "rel_time(sec)": "RelTime",
        "velocity(c/s)": "Value",
        }
    )
    

    print(df.columns)

    df = preprocess_data_rms(df)
    df = reducing_rows_by_mean_average(df, n_parameters=3, target_column="Value")

    return df

In [48]:
def plot_time_series(df, target_timestamp=None):
  """
  Plots a simple time series wi1th timestamps on the x-axis and data on the y-axis.
  Optionally adds a vertical line at a target timestamp.

  Args:
    df (pandas.DataFrame): DataFrame with 'time' and 'data' columns.
    target_timestamp (str or pd.Timestamp, optional): The timestamp to mark 
                                                      with a vertical line. 
                                                      Defaults to None.
  """

  plt.plot(df['RelTime'], df['Value'])
  plt.xlabel('Time')
  plt.ylabel('Data')
  plt.title('Time Series Plot')
  plt.xticks(rotation=45)  # Rotate x-axis labels for better readability

  # Add target timestamp line if provided
  if target_timestamp is not None:
    if isinstance(target_timestamp, str):
      target_timestamp = pd.Timestamp(target_timestamp)  # Convert to Timestamp if needed
    plt.axvline(x=target_timestamp, color='red', linestyle='--', label='Target Time')
    plt.legend()

  plt.show()

In [83]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt  # Import matplotlib for plotting

class EarthquakeDataLoader:
    def __init__(self, data, earthquake_timestamps, window_size, stride):
        """
        Initializes the EarthquakeDataLoader and generates all windows.

        Args:
            data (pd.DataFrame): DataFrame with 'AbsTime', 'RelTime', and 'Value' columns.
            earthquake_timestamps (list): List of timestamps for earthquakes.
            window_size (int): Size of the sliding window.
            stride (float): Percentage of the window size for the stride (0 < stride < 1).
        """
        self.data = data.copy()  # Create a copy of the DataFrame
        
        # Convert 'AbsTime' to datetime64[ns] if it's not already
        self.data['AbsTime'] = pd.to_datetime(self.data['AbsTime'])  
        
        self.data['AbsTime'] = self.data['AbsTime'].astype(np.int64)  # Convert 'AbsTime' to int64
        self.earthquake_timestamps = earthquake_timestamps
        self.window_size = window_size
        self.stride = stride
        self.all_windows = self.generate_windows()  # Generate windows immediately

    def generate_windows(self):
        """
        Generates all windows and targets.
        """
        # Ensure 'AbsTime' is in datetime format before using .dt accessor
        self.data['AbsTime'] = pd.to_datetime(self.data['AbsTime'])  
        
        self.data['day'] = self.data['AbsTime'].dt.date  # Add a 'day' column for grouping based on 'AbsTime'
        all_windows = []
        for day, group in self.data.groupby('day'):
            group_data = group['Value'].values  # Get 'Value' values for the current day

            stride = int(self.window_size * self.stride)  # Calculate stride as a percentage of window_size
            num_windows = int((len(group_data) - self.window_size) // stride + 1)  # Use calculated stride

            for i in range(num_windows):
                start_idx = int(i * stride)  # Use calculated stride
                end_idx = start_idx + self.window_size
                window_data = group_data[start_idx:end_idx]
                window_time = group['AbsTime'].iloc[start_idx:end_idx]  # Use 'AbsTime' for time reference

                # Find target index in the window
                target_index = self.find_target_index(window_time)

                # Create target sequence
                target_seq = np.zeros(self.window_size)
                if target_index is not None:
                    target_seq[target_index] = 1

                all_windows.append((window_data.reshape(-1, 1), target_seq.reshape(-1, 1)))

        return all_windows

    def find_target_index(self, window_time):
        """
        Finds the index of the closest timestamp to an earthquake in the window.

        Args:
            window_time (pd.Series): Series of 'AbsTime' timestamps in the current window.

        Returns:
            int or None: Index of the closest timestamp, or None if no earthquake is found.
        """
        for eq_time in self.earthquake_timestamps:
            closest_time = window_time[window_time.apply(lambda t: abs(t - eq_time).total_seconds() <= 1)]
            if not closest_time.empty:
                return closest_time.index[0] - window_time.index[0]
        return None

    def __getitem__(self, index):
        """
        Allows accessing windows using indexing.
        """
        return self.all_windows[index]

    def __len__(self):
        """
        Returns the total number of windows.
        """
        return len(self.all_windows)

    def save(self, filename):
        """
        Saves the dataloader to a pickle file.

        Args:
            filename (str): Name of the pickle file.
        """
        with open(filename, 'wb') as f:
            pickle.dump(self, f)

    @staticmethod
    def load(filename):
        """
        Loads the dataloader from a pickle file.

        Args:
            filename (str): Name of the pickle file.

        Returns:
            EarthquakeDataLoader: Loaded dataloader instance.
        """
        with open(filename, 'rb') as f:
            dataloader = pickle.load(f)
            dataloader.data['AbsTime'] = pd.to_datetime(dataloader.data['AbsTime'])  # Convert 'AbsTime' back to datetime
            return dataloader

    def __add__(self, other):
        """
        Combines two EarthquakeDataLoader instances.
        """
        if not isinstance(other, EarthquakeDataLoader):
            raise TypeError("Unsupported operand type for +: 'EarthquakeDataLoader' and '{}'".format(type(other)))

        combined_data = pd.concat([self.data, other.data])
        combined_earthquake_timestamps = self.earthquake_timestamps + other.earthquake_timestamps
        combined_dataloader = EarthquakeDataLoader(
            combined_data, 
            combined_earthquake_timestamps, 
            self.window_size, 
            self.stride
        )
        return combined_dataloader

In [70]:
df = pd.read_csv(r"..\data\lunar\test\data\S12_GradeB\xa.s12.00.mhz.1969-12-16HR00_evid00006.csv")

In [73]:
df = preprocess_df(df)

Index(['AbsTime', 'RelTime', 'Value'], dtype='object')


In [84]:
dataloader1 = QuakeDataLoader(df.head(30), quake_timestamps_pandas, 10, 0.25)

In [91]:
def create_and_save_dataloaders(lunar_data_file_paths, quake_timestamps_pandas, window_size, stride, output_folder):
    """
    Creates EarthquakeDataLoader instances for each CSV file and saves them as pickle files.

    Args:
        lunar_data_file_paths (list): List of paths to CSV files.
        quake_timestamps_pandas (list): List of earthquake timestamps.
        window_size (int): Size of the sliding window.
        stride (float): Percentage of the window size for the stride (0 < stride < 1).
        output_folder (str): Path to the folder where pickle files will be saved.
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)  # Create the output folder if it doesn't exist

    for csv_file in lunar_data_file_paths:
        try:
            df = pd.read_csv(csv_file)
            df = preprocess_df(df)  # Preprocess the data
            dataloader = EarthquakeDataLoader(df, quake_timestamps_pandas, window_size, stride)
            
            # Create output filename based on the CSV filename
            filename = os.path.splitext(os.path.basename(csv_file))[0] + ".pkl"
            output_path = os.path.join(output_folder, filename)
            
            dataloader.save(output_path)  # Save the dataloader as a pickle file
            print(f"DataLoader saved to: {output_path}")

        except Exception as e:
            print(f"Error processing {csv_file}: {e}")

In [ ]:
# Example usage:
lunar_data_file_paths = lunar_data_file_path
quake_timestamps_pandas = quake_timestamps_pandas
window_size = 11000
stride = 0.25
output_folder = r"..\data\lunar\training\pkl"  # Replace with your desired output folder

create_and_save_dataloaders(lunar_data_file_paths[16:], quake_timestamps_pandas, window_size, stride, output_folder)

Index(['AbsTime', 'RelTime', 'Value'], dtype='object')
DataLoader saved to: ..\data\lunar\training\pkl\xa.s12.00.mhz.1971-01-28HR00_evid00023.pkl
Index(['AbsTime', 'RelTime', 'Value'], dtype='object')
DataLoader saved to: ..\data\lunar\training\pkl\xa.s12.00.mhz.1971-01-29HR00_evid00024.pkl
Index(['AbsTime', 'RelTime', 'Value'], dtype='object')
DataLoader saved to: ..\data\lunar\training\pkl\xa.s12.00.mhz.1971-02-09HR00_evid00026.pkl
Index(['AbsTime', 'RelTime', 'Value'], dtype='object')
DataLoader saved to: ..\data\lunar\training\pkl\xa.s12.00.mhz.1971-03-25HR00_evid00028.pkl
Index(['AbsTime', 'RelTime', 'Value'], dtype='object')
DataLoader saved to: ..\data\lunar\training\pkl\xa.s12.00.mhz.1971-04-17HR00_evid00030.pkl
Index(['AbsTime', 'RelTime', 'Value'], dtype='object')
DataLoader saved to: ..\data\lunar\training\pkl\xa.s12.00.mhz.1971-05-12HR00_evid00031.pkl
Index(['AbsTime', 'RelTime', 'Value'], dtype='object')
DataLoader saved to: ..\data\lunar\training\pkl\xa.s12.00.mhz.1971-05